In [1]:
# Remove warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import needed libraries.
from crewai import Agent, Task, Crew

In [6]:
# Define our LLM.
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAI

load_dotenv()
api_key = os.environ['GOOGLE_API_KEY']


llm = GoogleGenerativeAI(model="models/gemini-pro", google_api_key=api_key)


## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [9]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    llm=llm,
    allow_delegation=False,
	verbose=True
)

In [10]:
planner

Agent(role=Content Planner, goal=Plan engaging and factually accurate content on {topic}, backstory=You're working on planning a blog article about the topic: {topic}.You collect information that helps the audience learn something and make informed decisions. Your work is the basis for the Content Writer to write an article on this topic.)

### Agent: Writer

In [11]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [12]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    llm=llm,    # Our llm object.
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [13]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [14]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [16]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [17]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],  # The order is important
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [18]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
**Content Plan: Artificial Intelligence**

**Executive Summary**

Artificial Intelligence (AI) is rapidly transforming industries, from healthcare and finance to manufacturing and transportation. This content plan will provide a comprehensive overview of the latest trends, key players, and noteworthy news in AI. The target audience for this blog article is business professionals and technology enthusiasts who want to learn more about AI and its potential impact. The content will be engaging, factually accurate

- Display the results of your execution as markdown in the notebook.

In [19]:
from IPython.display import Markdown
Markdown(result)

**

**Artificial Intelligence: The Next Technological Revolution**

**Introduction**

Artificial Intelligence (AI) is revolutionizing industries across the board, from healthcare and finance to manufacturing and transportation. With its ability to transform how we live, work, and connect with the world, AI holds immense potential for the future.

AI mimics intelligent human behavior, allowing machines to perform tasks typically requiring human intellect, such as learning, problem-solving, and decision-making. It's a field of computer science dedicated to creating systems that perform human-like cognitive functions.

**Key Trends in AI**

Several prominent trends shape the AI landscape:

* **Machine Learning and Deep Learning:** Machine learning enables computers to learn from data without explicit programming, while deep learning leverages artificial neural networks to identify complex patterns.
* **Natural Language Processing:** Computers can now comprehend and interact using human language, bridging the communication gap between humans and machines.
* **Computer Vision:** AI systems can analyze and interpret images and videos, enabling visual perception and understanding.
* **Robotics and Autonomous Systems:** AI plays a crucial role in controlling robots and autonomous systems, automating tasks and enhancing efficiency.

**Major Players in AI**

Leading technology giants are investing heavily in AI research and development, driving innovation in the field:

* **Google:** Focuses on advancements in machine learning, natural language processing, and computer vision.
* **Microsoft:** Invests in cloud-based AI platforms and applications for various industries.
* **Amazon:** Known for its Alexa AI assistant and cloud computing services for AI development.
* **IBM:** Specializes in enterprise AI solutions, including Watson for cognitive computing and healthcare applications.
* **Baidu:** A Chinese tech giant that excels in AI-powered search, language processing, and autonomous driving.

**Recent Developments in AI**

AI has witnessed significant advancements in recent months:

* Google AI's model generates realistic images from text descriptions, expanding creative possibilities.
* Microsoft's AI platform simplifies application development for programmers, accelerating AI adoption.
* Amazon's AI shopping assistant enhances customer experiences by personalizing recommendations and streamlining purchases.
* IBM Watson Health collaborates with the Mayo Clinic to develop AI tools for cancer diagnosis and treatment, improving patient outcomes.

**Call to Action**

As AI continues to rapidly evolve, staying informed about its latest advancements is crucial. We encourage you to:

* Delve into the resources provided in this blog post to deepen your understanding of AI.
* Engage in discussions about AI in the comments section, sharing your perspectives and experiences.
* Attend upcoming AI conferences and webinars to stay abreast of the latest developments.

By embracing AI's transformative potential, we can navigate its challenges and harness its benefits for a better future.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [20]:
topic = "Parallel Programming"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Parallel Programming.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
**Final Answer:**

**Content Plan: Parallel Programming**

**Introduction:**

* Define parallel programming and its significance in modern computing.
* Discuss the growing demand for parallelization and its applications across various industries.

**Key Points:**

* **Latest Trends in Parallel Programming:**
    * Multi-core and many-core architectures
    * Heterogeneous computing platforms (e.g., GPUs, FPGAs)
    * Cloud-based parallel computing services
* **Key Players in the Industry:**
    * Intel (OpenMP, TBB)
    * NVIDIA (CUDA)
    * AMD (HSA)
    *

In [21]:
Markdown(result)

# A Comprehensive Guide to Parallel Programming: Unlocking the Power of Modern Computing

## Introduction

Parallel programming has revolutionized modern computing by enabling complex tasks to be executed concurrently across multiple processing units. This approach enhances performance, efficiency, and scalability across a wide range of applications in scientific research, financial modeling, machine learning, and data analytics.

## Latest Trends and Advancements

The landscape of parallel programming is constantly evolving, driven by advancements in hardware and software technologies. Multi-core and heterogeneous computing platforms, cloud-based services, and specialized tools like OpenMP, CUDA, and C++ Amp empower developers to harness the power of parallelism with greater ease and efficiency.

## Benefits and Challenges

Parallel programming offers significant benefits such as improved performance, reduced development time, and increased scalability. However, it also presents challenges like data race conditions, deadlocks, and scalability issues. Careful design, synchronization primitives, and appropriate parallelization strategies are crucial to mitigate these challenges and maximize the benefits of parallel programming.

## Case Studies and Resources

Real-world applications demonstrate the transformative power of parallel programming. From scientific simulations to financial analysis and machine learning, parallel programming drives innovation and enables groundbreaking advancements. A wealth of resources, including online courses, tutorials, and books, supports developers in learning and implementing parallel programming techniques.

## Call to Action

Embrace parallel programming to unlock the full potential of modern computing. By understanding its principles, leveraging the latest advancements, and adopting best practices, developers can create applications that are faster, more scalable, and more efficient. Harness the power of parallelism to revolutionize your software solutions.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).